<a href="https://colab.research.google.com/github/kweenkeen/ReachOutandRead/blob/main/Mail_Merge_Monthly_Email_PROGRESS_REPORT_SEASON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import files
from datetime import date
from datetime import datetime

# UPLOAD SITE DATA REPORT

site_data_report = files.upload()
site_data_file = next(iter(site_data_report))
df = pd.read_excel(site_data_file, converters={'Site ID': str})
df = df.set_index('Site ID')

Saving MyROR Site Data Report 2022-08-16.xlsx to MyROR Site Data Report 2022-08-16.xlsx


In [2]:
# UPLOAD AFFILIATE CONTACT LIST

affiliate_contact_list = files.upload()
affiliate_filename = next(iter(affiliate_contact_list))
affiliate_contact_df = pd.read_excel(affiliate_filename)

Saving 2022.07.12 Find a Site Affiliate Contact Information.xlsx to 2022.07.12 Find a Site Affiliate Contact Information.xlsx


In [3]:
# UPLOAD CURRENT PROGRESS REPORT DOWNLOAD

current_progress_report = files.upload()
current_progress_report_file = next(iter(current_progress_report))
progress_df = pd.read_excel(current_progress_report_file, converters={'Site ID': str})
progress_df = progress_df.set_index('Site ID')

Saving MyROR Current progress report 2022-08-15.xlsx to MyROR Current progress report 2022-08-15.xlsx


In [6]:
# Set filter
active_or_approved = ['Active', 'Active - Non-Reporting',
                      'Slated for deactivation', 'COVID-19 Freeze',
                      'Approved - Needs Training', 'Approved - Waitlisted',
                      'In Progress'] 
df = df[df['Status'].isin(active_or_approved)]

rslt_df = df[['Affiliate Name', 'Status', 
              'Join Date', 'Training %', 'Total Trained', 
              'Most Recent Quality Classification Date', 
              'Primary Contact Email', 'Medical Consultant Email']]

mail_merge_df = pd.DataFrame(columns=['Affiliate Name'])
mail_merge_df = mail_merge_df.set_index('Affiliate Name')
mail_merge_df.head()

high_level_active = ['Active', 'Active - Non-Reporting', 
                     'Slated for deactivation', 'COVID-19 Freeze'] 
active_df = rslt_df[rslt_df['Status'].isin(high_level_active)] 
active_sites_df = active_df['Affiliate Name'].value_counts()
active_sites_df = active_sites_df.sort_index()
active_sites_df =active_sites_df.transpose()

mail_merge_df['Number of Sites (High Level Active)'] = active_sites_df

date_range_min = input("Type in the start of your year (MM/DD/YYYY): ")
date_range_min = datetime.strptime(date_range_min, '%m/%d/%Y')
date_range_max = input("Type in the end of your year (MM/DD/YYYY): ")
date_range_max = datetime.strptime(date_range_max, '%m/%d/%Y')

join_date_df = rslt_df.loc[(rslt_df['Join Date'] >= date_range_min) & (rslt_df['Join Date'] <= date_range_max)]
join_date_df['Join Date'] = join_date_df['Join Date'].dt.date

new_sites_df = join_date_df['Affiliate Name'].value_counts()
new_sites_df = pd.DataFrame({'Affiliate Name':new_sites_df.index, 'Number of New Sites':new_sites_df.values})
mail_merge_df = mail_merge_df.join(new_sites_df.set_index('Affiliate Name'), on='Affiliate Name')
mail_merge_df['Number of New Sites'] = mail_merge_df['Number of New Sites'].fillna(0)
mail_merge_df['Number of New Sites'] = mail_merge_df['Number of New Sites'].astype(int)

training_percent_average_df = active_df.groupby('Affiliate Name')['Training %'].mean()
training_percent_average_df = pd.DataFrame({'Affiliate Name':training_percent_average_df.index, 'Current Overall Training Percentage':training_percent_average_df.values})
mail_merge_df = mail_merge_df.join(training_percent_average_df.set_index('Affiliate Name'), on='Affiliate Name')
mail_merge_df['Current Overall Training Percentage'] = mail_merge_df['Current Overall Training Percentage']* 100
mail_merge_df['Current Overall Training Percentage'] = mail_merge_df['Current Overall Training Percentage'].astype(int)
mail_merge_df['Current Overall Training Percentage'] = mail_merge_df['Current Overall Training Percentage'].astype(str) + '%'

providers_trained_count_df = active_df.groupby('Affiliate Name')['Total Trained'].sum().astype(int)
providers_trained_count_df = pd.DataFrame({'Affiliate Name':providers_trained_count_df.index, 'Number of Providers Trained':providers_trained_count_df.values})
mail_merge_df = mail_merge_df.join(providers_trained_count_df.set_index('Affiliate Name'), on='Affiliate Name')

SQC_date_df = rslt_df.loc[(rslt_df['Most Recent Quality Classification Date'] >= date_range_min) & (rslt_df['Most Recent Quality Classification Date'] <= date_range_max)]
SQC_date_df['Most Recent Quality Classification Date'] = SQC_date_df['Most Recent Quality Classification Date'].dt.date
SQC_updates_df = SQC_date_df['Affiliate Name'].value_counts()
SQC_updates_df = pd.DataFrame({'Affiliate Name':SQC_updates_df.index, '# of SQCs Completed':SQC_updates_df.values})
mail_merge_df = mail_merge_df.join(SQC_updates_df.set_index('Affiliate Name'), on='Affiliate Name')
mail_merge_df['# of SQCs Completed'] = mail_merge_df['# of SQCs Completed'].fillna(0)
mail_merge_df['# of SQCs Completed'] = mail_merge_df['# of SQCs Completed'].astype(int)

in_progress_df = df[df['Status'] == 'In Progress'] 
in_progress_df = in_progress_df['Affiliate Name'].value_counts()
in_progress_df = pd.DataFrame({'Affiliate Name':in_progress_df.index, '# of Applications in Progress':in_progress_df.values})
mail_merge_df = mail_merge_df.join(in_progress_df.set_index('Affiliate Name'), on='Affiliate Name')
mail_merge_df['# of Applications in Progress'] = mail_merge_df['# of Applications in Progress'].fillna(0)
mail_merge_df['# of Applications in Progress'] = mail_merge_df['# of Applications in Progress'].astype(int)

approved_needs_training_df = df[df['Status'] == 'Approved - Needs Training'] 
approved_needs_training_df = approved_needs_training_df['Affiliate Name'].value_counts()
approved_needs_training_df = pd.DataFrame({'Affiliate Name':approved_needs_training_df.index, '# of Approved - Needs Training Sites':approved_needs_training_df.values})
mail_merge_df = mail_merge_df.join(approved_needs_training_df.set_index('Affiliate Name'), on='Affiliate Name')
mail_merge_df['# of Approved - Needs Training Sites'] = mail_merge_df['# of Approved - Needs Training Sites'].fillna(0)
mail_merge_df['# of Approved - Needs Training Sites'] = mail_merge_df['# of Approved - Needs Training Sites'].astype(int)

approved_waitlisted_df = df[df['Status'] == 'Approved - Waitlisted'] 
approved_waitlisted_df = approved_waitlisted_df['Affiliate Name'].value_counts()
approved_waitlisted_df = pd.DataFrame({'Affiliate Name':approved_waitlisted_df.index, '# of Approved - Waitlisted Sites':approved_waitlisted_df.values})
mail_merge_df = mail_merge_df.join(approved_waitlisted_df.set_index('Affiliate Name'), on='Affiliate Name')
mail_merge_df['# of Approved - Waitlisted Sites'] = mail_merge_df['# of Approved - Waitlisted Sites'].fillna(0)
mail_merge_df['# of Approved - Waitlisted Sites'] = mail_merge_df['# of Approved - Waitlisted Sites'].astype(int)

covid_freeze_df = df[df['Status'] == 'COVID-19 Freeze'] 
covid_freeze_df = covid_freeze_df['Affiliate Name'].value_counts()
covid_freeze_df = pd.DataFrame({'Affiliate Name':covid_freeze_df.index, '# of Covid Freeze Sites':covid_freeze_df.values})
mail_merge_df = mail_merge_df.join(covid_freeze_df.set_index('Affiliate Name'), on='Affiliate Name')
mail_merge_df['# of Covid Freeze Sites'] = mail_merge_df['# of Covid Freeze Sites'].fillna(0)
mail_merge_df['# of Covid Freeze Sites'] = mail_merge_df['# of Covid Freeze Sites'].astype(int)

S4D_df = df[df['Status'] == 'Slated for deactivation'] 
S4D_df = S4D_df['Affiliate Name'].value_counts()
S4D_df = pd.DataFrame({'Affiliate Name':S4D_df.index, '# of Slated for Deactivation Sites':S4D_df.values})
mail_merge_df = mail_merge_df.join(S4D_df.set_index('Affiliate Name'), on='Affiliate Name')
mail_merge_df['# of Slated for Deactivation Sites'] = mail_merge_df['# of Slated for Deactivation Sites'].fillna(0)
mail_merge_df['# of Slated for Deactivation Sites'] = mail_merge_df['# of Slated for Deactivation Sites'].astype(int)

contact_list1 = []
contact_list2 = []
contact_list3 = []
affiliate_list1 = []
affiliate_list2 = []
affiliate_list3 = []

for ind, row in active_df.iterrows():
  email_check1 = active_df['Primary Contact Email'][ind]
  email_check2 = active_df['Medical Consultant Email'][ind]

  # Check if there is NO  Primary Contact Email
  if type(email_check1) == float:
    contact_list1.append(ind)
    affiliate_list1.append(active_df.loc[ind, "Affiliate Name"])
  
  # Check if there is NO Medical Consultant Email
  if type(email_check2) == float:
    contact_list2.append(ind)
    affiliate_list2.append(active_df.loc[ind, "Affiliate Name"])

  # Check if there is NO Primary Contcat Email AND NO medical contact email
  if type(email_check1) == float and type(email_check2) == float:
    contact_list3.append(ind)
    affiliate_list3.append(active_df.loc[ind, "Affiliate Name"])

contact_df1 = pd.DataFrame({'Affiliate Name': pd.Series(affiliate_list1), 'Missing Primary Contact': pd.Series(contact_list1)})
contact_df2 = pd.DataFrame({'Affiliate Name': pd.Series(affiliate_list2), 'Missing Medical Contact': pd.Series(contact_list2)})
contact_df3 = pd.DataFrame({'Affiliate Name': pd.Series(affiliate_list3), 'Missing Primary Contact and Medical Contact': pd.Series(contact_list3)})

missing_primary_contact_df = contact_df1['Affiliate Name'].value_counts()
missing_primary_contact_df = pd.DataFrame({'Affiliate Name':missing_primary_contact_df.index, '# of Missing Primary Contacts':missing_primary_contact_df.values})
mail_merge_df = mail_merge_df.join(missing_primary_contact_df.set_index('Affiliate Name'), on='Affiliate Name')
mail_merge_df['# of Missing Primary Contacts'] = mail_merge_df['# of Missing Primary Contacts'].fillna(0)
mail_merge_df['# of Missing Primary Contacts'] = mail_merge_df['# of Missing Primary Contacts'].astype(int)

missing_medical_contact_df = contact_df2['Affiliate Name'].value_counts()
missing_medical_contact_df = pd.DataFrame({'Affiliate Name':missing_medical_contact_df.index, '# of Missing Medical Consultants':missing_medical_contact_df.values})
mail_merge_df = mail_merge_df.join(missing_medical_contact_df.set_index('Affiliate Name'), on='Affiliate Name')
mail_merge_df['# of Missing Medical Consultants'] = mail_merge_df['# of Missing Medical Consultants'].fillna(0)
mail_merge_df['# of Missing Medical Consultants'] = mail_merge_df['# of Missing Medical Consultants'].astype(int)


In [58]:
## ADDING IN INFORMAITON FOR PROGRESS REPORT UPDATES

submitted_progress_report_df = progress_df[progress_df['Current Report Status'] == 'Completed'] 
submitted_progress_report_df = submitted_progress_report_df['Affiliate'].value_counts()
submitted_progress_report_df = pd.DataFrame({'Affiliate Name':submitted_progress_report_df.index, 'Submitted Progress Reports':submitted_progress_report_df.values})
mail_merge_df = mail_merge_df.join(submitted_progress_report_df.set_index('Affiliate Name'), on='Affiliate Name')
mail_merge_df['Submitted Progress Reports'] = mail_merge_df['Submitted Progress Reports'].fillna(0)
mail_merge_df['Submitted Progress Reports'] = mail_merge_df['Submitted Progress Reports'].astype(int)

missing_progress_report_df = progress_df[progress_df['Current Report Status'] == 'Not Started'] 
missing_progress_report_df = missing_progress_report_df['Affiliate'].value_counts()
missing_progress_report_df = pd.DataFrame({'Affiliate Name':missing_progress_report_df.index, 'Missing Progress Reports':missing_progress_report_df.values})
mail_merge_df = mail_merge_df.join(missing_progress_report_df.set_index('Affiliate Name'), on='Affiliate Name')
mail_merge_df['Missing Progress Reports'] = mail_merge_df['Missing Progress Reports'].fillna(0)
mail_merge_df['Missing Progress Reports'] = mail_merge_df['Missing Progress Reports'].astype(int)

draft_progress_report_df = progress_df[progress_df['Current Report Status'] == 'Draft'] 
draft_progress_report_df = draft_progress_report_df['Affiliate'].value_counts()
draft_progress_report_df = pd.DataFrame({'Affiliate Name':draft_progress_report_df.index, 'Draft Progress Reports':draft_progress_report_df.values})
mail_merge_df = mail_merge_df.join(draft_progress_report_df.set_index('Affiliate Name'), on='Affiliate Name')
mail_merge_df['Draft Progress Reports'] = mail_merge_df['Draft Progress Reports'].fillna(0)
mail_merge_df['Draft Progress Reports'] = mail_merge_df['Draft Progress Reports'].astype(int)

mail_merge_df['% of Submitted Progress Reports'] = mail_merge_df['Submitted Progress Reports']/mail_merge_df['Number of Sites (High Level Active)']
mail_merge_df['% of Submitted Progress Reports'] = mail_merge_df['% of Submitted Progress Reports']* 100
mail_merge_df['% of Submitted Progress Reports'] = mail_merge_df['% of Submitted Progress Reports'].astype(int)
mail_merge_df['% of Submitted Progress Reports'] = mail_merge_df['% of Submitted Progress Reports'].astype(str) + '%'

In [70]:
mail_merge_df = mail_merge_df.join(affiliate_contact_df.set_index('Affiliate Name'), on='Affiliate Name')

mail_merge_df.rename(index={'Reach Out and Read Florida (excluding Palm Beach County)':'Reach Out and Read Florida'},inplace=True)
mail_merge_df.rename(index={'Reach Out and Read Inland Empire (California)':'Reach Out and Read Inland Empire'},inplace=True)
mail_merge_df.rename(index={'Reach Out and Read Orange County (California)':'Reach Out and Read Orange County'},inplace=True)
mail_merge_df.rename(index={'Reach Out and Read Palm Beach County (Florida)':'Reach Out and Read Palm Beach County'},inplace=True)

# SAVE RESULTS DATAFRAME AS EXCEL FILE 
today = str(date.today())
excel_filename = "Mail_Merge_" + today + ".xlsx"
mail_merge_df.to_excel(excel_filename)
files.download(excel_filename)